# Network analysis

This generates the three inputs for network analysis and calls the algorithm. This script only runs the analysis on the key genes rather than the whole differentially expressed gene list, in the interest of time.

In [8]:
import pandas as pd

Run this in the remote terminal:
- screen
- ipcluster start -n 3

In [11]:
import ipyparallel as ipp
client = ipp.Client()
view = client.load_balanced_view()
print("Connected engines:", len(client.ids))

Connected engines: 3


# Drug proximity analysis

## Define functions
The code from this chunk is taken from a paper by Han et al. (2021) titled "Identification of SARS-CoV-2–induced pathways reveals drug repurposing strategies".

In [2]:
# functions
def has_path(G,node_from,node_to): # returns true if the graph G has a path from source node(node_from) to target node (node_to)
    return(nx.has_path(G,node_from,node_to))


def get_shortest_path_length_between(G, source_id, target_id): #calculates shortest path length between source and target nodes in the graph G
    return nx.shortest_path_length(G, source_id, target_id) 

'''
The function below uses the previous two functions to calculate the path length 
between each source node (drug target) and its closest disease target.
See Guney 2015 for more details on the approach (Network-based in silico drug efficacy screening).
'''
def calculate_closest_distance(network, nodes_from, nodes_to):
    values_outer = []
    for node_from in nodes_from: #nodes_from is a list of drug targets (eg. from drugbank, chembl, etc.)
        values = [] # will store the shortest path length between a source node and all disease targets here
        for node_to in nodes_to: #nodes_to is a list of known disease targets
            # print("from - to", node_from, node_to)
            if not has_path(network,node_from,node_to): continue
            val = get_shortest_path_length_between(network, node_from, node_to)
            values.append(val)
        if len(values) == 0:    continue
        d = min(values) # the shortest path between a source node and its closest disease target
        # print (d)
        values_outer.append(d)
    closest_d = numpy.mean(values_outer) # the average shortest path length between any source node (drug target) and its closest disease target
    # print (d)
    return closest_d


def get_degree_binning(g, bin_size):
    '''
    This function creates the bins from a network g.
    Starting from a list of nodes with the lowest degree, it adds nodes with the same degree to the bin until it reaches the set bin size.
    If number of nodes with some degree is lower then bin size, it combines with other nodes with degree + 1 to meet bin size.
    '''
    degree_to_nodes = {}
    # the below two lines compute the degree of each node in the graph.
    # the setdefault method is used to add each node to a list of nodes with the same degree in the dictionary
    for node, degree in g.degree(): 
        degree_to_nodes.setdefault(degree, []).append(node)
    values = degree_to_nodes.keys()
    values = sorted(values) # values becomes a list, sorted from lowest to highest degree
    bins = []
    i = 0 # this is the iterator that iterates over each degree, starting from the first item in the list (lowest degree)
    while i < len(values):
        low = values[i] # this is the i-th degree in the values list
        val = degree_to_nodes[values[i]] # a list of the nodes with i-th degree (low)
        while len(val) < bin_size:
            # while the number of nodes in a bin is lower than the bin size, than nodes with degree i+1 will be added to the bin
            # bin size is chosen by the user - in the paper this is set to 100
            i += 1 # next iteration (move to the next degree in the list)
            if i == len(values): # breaks when the last item in the list is reached
                break
            # starting from a list of nodes with the lowest degree, it adds nodes with degree lowest + 1 to the val list until it reaches the set bin size
            val.extend(degree_to_nodes[values[i]]) # val will be extended with the next set of nodes with degree i+1 (low +1).
        if i == len(values):
            i -= 1
        high = values[i] # this is the highest degree
        i += 1
        # print i, low, high, len(val)
        if len(val) < bin_size:
            low_, high_, val_ = bins[-1]
            bins[-1] = (low_, high, val_ + val)
        else:
            bins.append((low, high, val))
    return bins

# this function lists all nodes in the same bin as each seed node
def get_degree_equivalents(seeds, bins, g): 
    seed_to_nodes = {}
    for seed in seeds:
        d = g.degree(seed) #extract degree of the seed node
        for l, h, nodes in bins: #it takes low, high degree and nodes for each bin
            if l <= d and h >= d:
                mod_nodes = list(nodes)
                mod_nodes.remove(seed)
                seed_to_nodes[seed] = mod_nodes
                break
    return seed_to_nodes

    
def pick_random_nodes_matching_selected(network, bins, nodes_selected, n_random, degree_aware=True, connected=False,
                                        seed=None):
    """
    Use get_degree_binning to get bins
    """
    if seed is not None:
        random.seed(seed)
    values = []
    nodes = network.nodes() # list of nodes in the network
    for i in range(n_random): # decided by the user (how many times will the random iterations be repeated?) usually this is = 1000
        if degree_aware:
            if connected:
                raise ValueError("Not implemented!")
            # the lines below pick random nodes matching the degree (same bin) of the real nodes
            nodes_random = set()
            node_to_equivalent_nodes = get_degree_equivalents(nodes_selected, bins, network) # lists nodes in the same bin as the node of interest
            # now choose a random node from the same bin as the real node
            for node, equivalent_nodes in node_to_equivalent_nodes.items():
                chosen = random.choice(equivalent_nodes)
                for k in range(20):  # Try to find a distinct node (at most 20 times) - to make sure it doesn't choose the same node
                    if chosen in nodes_random:
                        chosen = random.choice(equivalent_nodes)
                nodes_random.add(chosen)
            nodes_random = list(nodes_random)
        else:
            if connected:
                nodes_random = [random.choice(nodes)]
                k = 1
                while True:
                    if k == len(nodes_selected):
                        break
                    node_random = random.choice(nodes_random)
                    node_selected = random.choice(network.neighbors(node_random))
                    if node_selected in nodes_random:
                        continue
                    nodes_random.append(node_selected)
                    k += 1
            else:
                nodes_random = random.sample(nodes, len(nodes_selected))
        values.append(nodes_random)
    return values

def get_random_nodes(nodes, network, bins=None, n_random=1000, min_bin_size=100, degree_aware=True, seed=None):
    '''
    This function creates a n_random number of lists of random nodes with the same degree binning as the real nodes (when degree_aware=True).
    usually n_random = 1000 because we often do 1000 iterations.
    '''
    if bins is None:
        # Get degree bins of the network (if they aren't already supplied
        bins = get_degree_binning(network, min_bin_size)
    # pick the random nodes
    nodes_random = pick_random_nodes_matching_selected(network, bins, nodes, n_random, degree_aware,
                                                                         seed=seed)
    return nodes_random

def calculate_proximity(network, drug, nodes_from, nodes_to, nodes_from_random=None, nodes_to_random=None, bins=None,
                        n_random=1000, min_bin_size=100, seed=452456):
    """
    Calculate proximity from nodes_from to nodes_to
    If degree binning or random nodes are not given, they are generated
    """

    nodes_network = set(network.nodes())
    nodes_from = set(nodes_from) & nodes_network # select only nodes_from (drug targets) that are located in the network
    nodes_to = set(nodes_to) & nodes_network # select only nodes_to (disease targets) that are located in the network
    if len(nodes_from) == 0 or len(nodes_to) == 0:
        return None  # At least one of the node group not in network
    d = calculate_closest_distance(network, nodes_from, nodes_to) # this is the real distance
    
    # now do 1000 iterations using random nodes
    if bins is None and (nodes_from_random is None or nodes_to_random is None):
        bins = get_degree_binning(network, min_bin_size)
    if nodes_from_random is None:
        nodes_from_random = get_random_nodes(nodes_from, network, bins=bins, n_random=n_random,
                                             min_bin_size=min_bin_size, seed=seed)
    if nodes_to_random is None:
        nodes_to_random = get_random_nodes(nodes_to, network, bins=bins, n_random=n_random, min_bin_size=min_bin_size,
                                           seed=seed)
    random_values_list = zip(nodes_from_random, nodes_to_random)
    values = numpy.empty(len(nodes_from_random))  # n_random
    # now calculates the closest distance using random nodes. Repeat x1000
    for i, values_random in enumerate(random_values_list):
        #print('iteration ', i)
        nodes_from, nodes_to = values_random
        values[i] = calculate_closest_distance(network, nodes_from, nodes_to)
    m, s = numpy.mean(values), numpy.std(values) # do mean and stdev of random iterations
    if s == 0:
        z = 0.0
    else:
        z = (d - m) / s
    dict = {'drug': drug, 'distance': d, 'z_score': z}
    return dict

## Call function

In [3]:
import networkx as nx
import pandas as pd
import sys, time
import numpy
import random

In [4]:
# Create PPI network
ppi_df = pd.read_csv("../data/networks/combined_PPI.csv", sep=",")
ppi_graph = nx.from_pandas_edgelist(ppi_df, "GeneA", "GeneB")

# Get degree bins
bins = get_degree_binning(ppi_graph, bin_size=100)

# Print basic information about the graph
print(f"Number of nodes in PPI graph: {ppi_graph.number_of_nodes()}")
print(f"Number of edges in PPI graph: {ppi_graph.number_of_edges()}")

Number of nodes in PPI graph: 18451
Number of edges in PPI graph: 345547


In [5]:
# Prepare DPI data
dpi_df = pd.read_csv("../data/networks/combined_DPI.csv", sep=",")

# Drop NA
dpi_df.dropna(inplace=True)

# Filter DPI to keep only drug targets that are in the PPI graph
dpi_df = dpi_df[dpi_df["Drug_Target"].isin(ppi_graph.nodes)]
drug_list = dpi_df["Drug_Name"].unique()

# Print basic information about the DPI
print(f"Number of drug targets retained in DPI: {len(drug_list)}")

# Create a mapping of drug names to their targets
drug_to_targets = dpi_df.groupby("Drug_Name")["Drug_Target"].apply(set).to_dict()

Number of drug targets retained in DPI: 2244


### Define function for proximity analysis

In [6]:
def run_proximity_analysis(step_name, deg_file_path, output_csv_path, ppi_graph, drug_to_targets,
                           bins, n_random=1000, min_bin_size=100, seed=452456):
    """
    Calculate proximity for step_name using drugs and targets in drug_to_targets and disease genes in deg_file_path
    Saves the result as a CSV file at output_csv_path
    """
    print(f"\n===== Processing {step_name} =====")
    
    # ========== LOAD DISEASE GENES ==========
    disease_genes = pd.read_csv(deg_file_path, header=None)
    print(len(disease_genes), f"disease genes in {step_name}")
    disease_genes = disease_genes.iloc[:,0].dropna().unique().tolist()
    # Keep only disease genes that are in the PPI graph
    disease_genes = [gene for gene in disease_genes if gene in ppi_graph.nodes]
    print(len(disease_genes), f"disease genes in {step_name} after filtering for PPI graph")
    
    # ========== GENERATE RANDOM SETS OF DISEASE GENES ==========
    # For each disease gene, pick a random node from the same degree bin, and repeat this 1000 times, generating 1000 sets of random disease genes
    random_disease_genes = get_random_nodes(
        disease_genes,
        ppi_graph,
        bins=bins,
        n_random=n_random,
        min_bin_size=min_bin_size,
        seed=seed,
        degree_aware=True)
    print("Generated", len(random_disease_genes), "sets of random disease genes")

    # ========== RUN PROXIMITY ANALYSIS ==========
    results = []
    # Iterate over each drug and calculate proximity to disease genes
    for i, (drug, targets) in enumerate(drug_to_targets.items()):
        start = time.time()
        print(f"Processing drug: {drug}")

        # Calculate proximity to disease genes
        result = calculate_proximity(
            ppi_graph,
            drug,
            targets,
            disease_genes,
            nodes_from_random=random_disease_genes,
            bins=bins,
            n_random=n_random,
            min_bin_size=min_bin_size,
            seed=seed
        )

        if result is not None:
            results.append(result)
        else:
            print(f"No proximity data for drug: {drug}")

        if i % 50 == 0:
            print(f"{i}/{len(drug_to_targets)} drugs processed")

        end = time.time()
        print(f"Runtime: {(end - start):.2f} sec.")

    # Save and display
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv_path, index=False)
    print(f"Saved results to {output_csv_path}")
    print("Top 10 prioritised drugs for", step_name, results_df.sort_values("z_score").head(10))


### Call function for each step

In [7]:
# Define DEG file paths and output paths
steps = {
    "step1": {
        "deg_file": "../results/humanPVATsn/network_analysis/step1_key_genes_only.csv",
        "output_csv": "../results/humanPVATsn/network_analysis/proximity_step1_key_only.csv"
    },
    "step2": {
        "deg_file": "../results/humanPVATsn/network_analysis/step2_key_genes_only.csv",
        "output_csv": "../results/humanPVATsn/network_analysis/proximity_step2_key_only.csv"
    },
    "step3": {
        "deg_file": "../results/humanPVATsn/network_analysis/step3_key_genes_only.csv",
        "output_csv": "../results/humanPVATsn/network_analysis/proximity_step3_key_only.csv"
    },
    "full": {
        "deg_file": "../results/humanPVATsn/network_analysis/full_key_genes_only.csv",
        "output_csv": "../results/humanPVATsn/network_analysis/proximity_full_key_only.csv"
    }
}

# Run proximity analysis for each step
# for step, path in steps.items(): CHANGE THIS BACK AFTERWARDS – I HAD A TYPO IN STEP 2 AND DID NOT WANT TO RERUN STEP 1
# for step in ["step2", "step3", "full"]: # DELETE THIS LINE AFTERWARDS
#    path = steps[step] # DELETE THIS LINE AFTERWARDS
 #   run_proximity_analysis(step_name=step,
 #                          deg_file_path=path["deg_file"],
  #                         output_csv_path=path["output_csv"],
   #                        ppi_graph=ppi_graph,
    #                       drug_to_targets=drug_to_targets,
     #                      bins=bins,
      #                     n_random=100,
       #                    min_bin_size=100,
        #                   seed=452456)

Runtime: 8.15 sec.
Processing drug: hydroquinone
Runtime: 9.16 sec.
Processing drug: hydroxocobalamin
Runtime: 6.84 sec.
Processing drug: hydroxyamphetamine
Runtime: 6.62 sec.
Processing drug: hydroxyamphetamine hydrobromide
Runtime: 7.45 sec.
Processing drug: hydroxychloroquine
Runtime: 8.84 sec.
Processing drug: hydroxyurea
Runtime: 7.28 sec.
Processing drug: hydroxyzine
Runtime: 7.98 sec.
Processing drug: hyoscyamine
Runtime: 7.36 sec.
Processing drug: ibalizumab
Runtime: 8.76 sec.
Processing drug: ibandronate
Runtime: 7.62 sec.
Processing drug: ibandronic acid
Runtime: 8.34 sec.
Processing drug: ibritumomab tiuxetan
Runtime: 9.40 sec.
Processing drug: ibrutinib
Runtime: 8.87 sec.
Processing drug: ibuprofen
Runtime: 7.43 sec.
Processing drug: ibutilide
Runtime: 7.41 sec.
Processing drug: icatibant
Runtime: 10.58 sec.
Processing drug: idarubicin
Runtime: 7.14 sec.
Processing drug: idarubicin hydrochloride
Runtime: 6.95 sec.
Processing drug: idebenone
Runtime: 7.02 sec.
Processing dru

Runtime: 8.59 sec.
Processing drug: levothyroxine
Runtime: 8.57 sec.
Processing drug: lidocaine
Runtime: 11.23 sec.
Processing drug: lidoflazine
Runtime: 8.13 sec.
Processing drug: lifitegrast
Runtime: 7.15 sec.
Processing drug: linaclotide
Runtime: 5.99 sec.
Processing drug: linagliptin
Runtime: 8.88 sec.
Processing drug: lincomycin
Runtime: 8.85 sec.
Processing drug: lindane
Runtime: 8.38 sec.
Processing drug: linezolid
Runtime: 7.42 sec.
Processing drug: linzagolix
Runtime: 7.10 sec.
Processing drug: liothyronine
Runtime: 7.85 sec.
Processing drug: liothyronine sodium
Runtime: 6.87 sec.
Processing drug: liotrix
Runtime: 7.02 sec.
Processing drug: lipegfilgrastim
Runtime: 7.52 sec.
Processing drug: lipoic acid, alpha
Runtime: 8.23 sec.
Processing drug: liraglutide
Runtime: 9.10 sec.
Processing drug: lisdexamfetamine
Runtime: 9.12 sec.
Processing drug: lisinopril
Runtime: 7.12 sec.
Processing drug: lisinopril anhydrous
Runtime: 6.62 sec.
Processing drug: lisocabtagene maraleucel
Runti

Runtime: 5.02 sec.
Processing drug: metyrosine
Runtime: 4.96 sec.
Processing drug: mexiletine
Runtime: 4.96 sec.
Processing drug: mexiletine hydrochloride
Runtime: 4.90 sec.
Processing drug: mianserin
Runtime: 5.02 sec.
Processing drug: mibefradil
Runtime: 5.04 sec.
Processing drug: mibefradil dihydrochloride
Runtime: 5.01 sec.
Processing drug: micafungin
Runtime: 6.52 sec.
Processing drug: miconazole
Runtime: 7.92 sec.
Processing drug: miconazole nitrate
Runtime: 6.76 sec.
Processing drug: midazolam
Runtime: 6.70 sec.
Processing drug: midodrine
Runtime: 7.04 sec.
Processing drug: midostaurin
Runtime: 5.89 sec.
Processing drug: mifamurtide
Runtime: 5.19 sec.
Processing drug: mifepristone
Runtime: 5.33 sec.
Processing drug: migalastat
Runtime: 5.27 sec.
Processing drug: miglitol
Runtime: 5.13 sec.
Processing drug: miglustat
Runtime: 5.24 sec.
Processing drug: milnacipran
Runtime: 5.24 sec.
Processing drug: milrinone
Runtime: 5.34 sec.
Processing drug: miltefosine
Runtime: 5.32 sec.
Proc

Runtime: 10.27 sec.
Processing drug: olutasidenib
Runtime: 7.25 sec.
Processing drug: omalizumab
Runtime: 8.07 sec.
Processing drug: omeprazole
Runtime: 7.26 sec.
Processing drug: omidenepag isopropyl
Runtime: 6.93 sec.
Processing drug: onapristone
Runtime: 6.82 sec.
Processing drug: ondansetron
Runtime: 6.89 sec.
Processing drug: opicapone
Runtime: 6.73 sec.
Processing drug: orciprenaline
Runtime: 6.91 sec.
Processing drug: orlistat
1500/2244 drugs processed
Runtime: 6.71 sec.
Processing drug: orphenadrine
Runtime: 7.20 sec.
Processing drug: oseltamivir
Runtime: 7.02 sec.
Processing drug: oseltamivir phosphate
Runtime: 8.00 sec.
Processing drug: osilodrostat
Runtime: 6.26 sec.
Processing drug: osimertinib
Runtime: 6.69 sec.
Processing drug: ospemifene
Runtime: 7.15 sec.
Processing drug: oteseconazole
Runtime: 7.48 sec.
Processing drug: ouabain
Runtime: 6.32 sec.
Processing drug: oxacillin
Runtime: 6.23 sec.
Processing drug: oxandrolone
Runtime: 7.58 sec.
Processing drug: oxaprozin
Run

Runtime: 5.51 sec.
Processing drug: polymyxin b
Runtime: 5.36 sec.
Processing drug: polythiazide
Runtime: 5.39 sec.
Processing drug: pomalidomide
Runtime: 5.34 sec.
Processing drug: ponatinib
Runtime: 5.76 sec.
Processing drug: ponesimod
Runtime: 5.80 sec.
Processing drug: posaconazole
Runtime: 5.62 sec.
Processing drug: potassium bromide
Runtime: 7.02 sec.
Processing drug: potassium iodide
Runtime: 5.31 sec.
Processing drug: pozelimab
Runtime: 5.27 sec.
Processing drug: practolol
Runtime: 5.30 sec.
Processing drug: pralatrexate
Runtime: 5.30 sec.
Processing drug: pralidoxime
Runtime: 5.26 sec.
Processing drug: pralidoxime chloride
Runtime: 5.43 sec.
Processing drug: pralsetinib
Runtime: 5.31 sec.
Processing drug: pramipexole
Runtime: 5.35 sec.
Processing drug: pramipexole dihydrochloride
Runtime: 5.37 sec.
Processing drug: pramlintide
Runtime: 5.35 sec.
Processing drug: pramoxine
Runtime: 5.34 sec.
Processing drug: prasterone
Runtime: 5.29 sec.
Processing drug: prasugrel
Runtime: 5.26

Runtime: 6.08 sec.
Processing drug: rucaparib
Runtime: 5.94 sec.
Processing drug: rufinamide
Runtime: 7.70 sec.
Processing drug: rupatadine
Runtime: 7.46 sec.
Processing drug: rurioctocog alfa pegol
Runtime: 7.25 sec.
Processing drug: rutin
Runtime: 6.09 sec.
Processing drug: ruxolitinib
Runtime: 5.67 sec.
Processing drug: ruxolitinib phosphate
Runtime: 7.18 sec.
Processing drug: sacituzumab govitecan
Runtime: 7.46 sec.
Processing drug: sacubitril
Runtime: 5.47 sec.
Processing drug: safinamide
Runtime: 5.33 sec.
Processing drug: salicylic acid
Runtime: 5.12 sec.
Processing drug: salmeterol
Runtime: 5.14 sec.
Processing drug: salmeterol xinafoate
Runtime: 5.65 sec.
Processing drug: salmon calcitonin
Runtime: 6.05 sec.
Processing drug: salsalate
Runtime: 7.77 sec.
Processing drug: samidorphan
Runtime: 6.73 sec.
Processing drug: sapropterin
Runtime: 5.17 sec.
Processing drug: saquinavir
Runtime: 5.20 sec.
Processing drug: saralasin
Runtime: 5.18 sec.
Processing drug: sargramostim
Runtime:

Runtime: 7.59 sec.
Processing drug: telotristat etiprate
Runtime: 7.67 sec.
Processing drug: temazepam
Runtime: 7.89 sec.
Processing drug: temozolomide
Runtime: 6.69 sec.
Processing drug: temsirolimus
Runtime: 6.49 sec.
Processing drug: tenapanor
Runtime: 6.29 sec.
Processing drug: tenapanor hydrochloride
Runtime: 6.14 sec.
Processing drug: tenecteplase
Runtime: 7.27 sec.
Processing drug: teniposide
Runtime: 8.33 sec.
Processing drug: tenoxicam
Runtime: 7.42 sec.
Processing drug: teplizumab
Runtime: 8.22 sec.
Processing drug: tepotinib
Runtime: 7.52 sec.
Processing drug: teprotumumab
Runtime: 6.45 sec.
Processing drug: terazosin
Runtime: 6.36 sec.
Processing drug: terbinafine
Runtime: 5.72 sec.
Processing drug: terbutaline
Runtime: 5.95 sec.
Processing drug: terconazole
Runtime: 5.75 sec.
Processing drug: terfenadine
Runtime: 7.13 sec.
Processing drug: teriflunomide
Runtime: 9.43 sec.
Processing drug: teriparatide
2000/2244 drugs processed
Runtime: 8.81 sec.
Processing drug: terlipress

Runtime: 5.28 sec.
Processing drug: tyloxapol
Runtime: 5.80 sec.
Processing drug: ublituximab
Runtime: 5.68 sec.
Processing drug: ubrogepant
Runtime: 5.04 sec.
Processing drug: udenafil
Runtime: 5.23 sec.
Processing drug: ulipristal
Runtime: 6.18 sec.
Processing drug: ulipristal acetate
Runtime: 7.06 sec.
Processing drug: ulobetasol
Runtime: 5.73 sec.
Processing drug: umbralisib
2150/2244 drugs processed
Runtime: 5.28 sec.
Processing drug: umeclidinium
Runtime: 4.96 sec.
Processing drug: umeclidinium bromide
Runtime: 4.92 sec.
Processing drug: upadacitinib
Runtime: 4.90 sec.
Processing drug: urea
Runtime: 4.85 sec.
Processing drug: ursodeoxycholic acid
Runtime: 4.85 sec.
Processing drug: ursodiol
Runtime: 4.87 sec.
Processing drug: ustekinumab
Runtime: 4.80 sec.
Processing drug: vaborbactam
Runtime: 4.98 sec.
Processing drug: vadadustat
Runtime: 4.97 sec.
Processing drug: valacyclovir
Runtime: 4.93 sec.
Processing drug: valbenazine
Runtime: 5.24 sec.
Processing drug: valdecoxib
Runtime

FileNotFoundError: [Errno 2] No such file or directory: '.../results/humanPVATsn/network_analysis/step2_key_genes_only.csv'

In [ ]:
path2 = steps["step2"]
run_proximity_analysis(step_name=step,
                           deg_file_path=path2["deg_file"],
                           output_csv_path=path2["output_csv"],
                           ppi_graph=ppi_graph,
                           drug_to_targets=drug_to_targets,
                           bins=bins,
                           n_random=100,
                           min_bin_size=100,
                           seed=452456)

In [ ]:
path3 = steps["step3"]
run_proximity_analysis(step_name=step,
                           deg_file_path=path3["deg_file"],
                           output_csv_path=path3["output_csv"],
                           ppi_graph=ppi_graph,
                           drug_to_targets=drug_to_targets,
                           bins=bins,
                           n_random=100,
                           min_bin_size=100,
                           seed=452456)

In [ ]:
path_full = steps["full"]
run_proximity_analysis(step_name=step,
                           deg_file_path=path_full["deg_file"],
                           output_csv_path=path_full["output_csv"],
                           ppi_graph=ppi_graph,
                           drug_to_targets=drug_to_targets,
                           bins=bins,
                           n_random=100,
                           min_bin_size=100,
                           seed=452456)